In [1]:
from genfft import generate_fft, default_config, fft_two_factor_twiddle
import numpy as np
from copy import copy

In [2]:
cfg = copy(default_config)
cfg["generator_path"] = "../genfft"
cfg["compile_args"] = ["-x", "c++", "-O3", "-include", "genfft/codelet.hh", "-shared"]

In [3]:
fft12 = fft_two_factor_twiddle(cfg, 3, 4)
x = np.arange(12, dtype='complex64')
fft12(x) - np.fft.fft(x)

array([0.+0.00000000e+00j, 0.-1.37861639e-06j, 0.-4.24942073e-07j,
       0.+0.00000000e+00j, 0.-5.39011656e-07j, 0.+4.24942072e-07j,
       0.+8.88178420e-16j, 0.-4.24942073e-07j, 0.+5.39011656e-07j,
       0.+8.88178420e-16j, 0.+4.24942073e-07j, 0.+1.37861639e-06j])

In [4]:
fft27 = fft_two_factor_twiddle(cfg, 3, 9)
x = np.arange(27, dtype='complex64')
np.abs(fft27(x) - np.fft.fft(x)).max()

8.962143078707292e-06

In [5]:
N = 3
fftN = generate_fft(cfg, "notw_complex", n=N)
x = np.arange(N, dtype='complex64')
y = np.zeros_like(x)
fftN(x, y) 
y

notw 3 1 [2] [2]


array([ 3. +0.j       , -1.5+0.8660254j, -1.5-0.8660254j], dtype=complex64)

In [6]:
def w(k, n):
    return np.exp(2j * np.pi * k / n)

def make_twiddle(n1, n2):
    I1 = np.arange(n1)
    I2 = np.arange(n2)
    return w(I1[:,None] * I2[None,:], n1*n2).astype('complex64')

def make_fft(config, n, m):
    fft_n = generate_fft(config, "notw_complex", n=n)
    fft_m = generate_fft(config, "twiddle_complex", n=m)
    W = make_twiddle(n, m)[:,1:].copy()
    W1 = make_twiddle(m, n).conj()

    def fft1(x):
        y = np.zeros_like(x).reshape([m, n])
        z = np.zeros_like(x)
        fft_n(x.reshape([n, m]).T, y)
        y *= W1
        fft_m(y.T, z.reshape([m, n]).T)
        return z

    def fft(x):
        y = np.zeros_like(x).reshape([m, n])
        fft_n(x.reshape([n, m]).T, y)
        fft_m(y.T, W)
        return y.flatten()

    return fft

In [7]:
import subprocess

try:
    fft12 = make_fft(cfg, 3, 4)
except subprocess.CalledProcessError as e:
    print(e.stderr)

Uncaught error running job: 140240412823184, Command '['g++', '-x', 'c++', '-O3', '-include', 'genfft/codelet.hh', '-shared', '-o', 'lib/30c8ab0d0557472a89f19e24e6970a93.so', '-']' returned non-zero exit status 1.
Flushing queue and waiting for threads to close.


Exception of type <class 'subprocess.CalledProcessError'> :
Command '['g++', '-x', 'c++', '-O3', '-include', 'genfft/codelet.hh', '-shared', '-o', 'lib/30c8ab0d0557472a89f19e24e6970a93.so', '-']' returned non-zero exit status 1.
<stdin>:9:14: fout: ‘__constant’ has not been declared
<stdin>:9:27: fout: expected ‘,’ or ‘...’ before ‘*’ token
<stdin>: In functie ‘void fft(R*, int)’:
<stdin>:13:14: fout: ‘mb’ was not declared in this scope; did you mean ‘m’?
<stdin>:13:18: fout: ‘W’ was not declared in this scope
<stdin>:13:40: fout: ‘me’ was not declared in this scope; did you mean ‘m’?
<stdin>:14:31: fout: ‘ms’ was not declared in this scope; did you mean ‘m’?
<stdin>:14:71: fout: ‘rs’ was not declared in this scope; did you mean ‘ri’?
<stdin>:16:12: fout: ‘ii’ was not declared in this scope; did you mean ‘ri’?



In [8]:
x = np.arange(12, dtype='complex64')
fft12(x).reshape([4, 3])

array([[66. +0.j       , -6.+22.392303j , -6.+10.392304j ],
       [-6. +6.j       , -6. +3.464101j , -6. +1.6076956j],
       [-6. +0.j       , -6. -1.6076956j, -6. -3.464101j ],
       [-6. -6.j       , -6.-10.392304j , -6.-22.392303j ]],
      dtype=complex64)

In [9]:
np.fft.fft(x).astype('complex64')

array([66.+0.0000000e+00j, -6.+2.2392305e+01j, -6.+1.0392304e+01j,
       -6.+6.0000000e+00j, -6.+3.4641016e+00j, -6.+1.6076951e+00j,
       -6.-8.8817842e-16j, -6.-1.6076951e+00j, -6.-3.4641016e+00j,
       -6.-6.0000000e+00j, -6.-1.0392304e+01j, -6.-2.2392305e+01j],
      dtype=complex64)